In [1]:
!pip install kss

In [2]:
import pickle


In [3]:
with open('economy_id.pickle', 'rb') as f:
    id_news = pickle.load(f)
with open('politic_id.pickle', 'rb') as f:
    id_news.extend(pickle.load(f))
with open('society_id.pickle', 'rb') as f:
    id_news.extend(pickle.load(f))

In [4]:

with open('economy_contents.pickle', 'rb') as f:
    news = pickle.load(f)
with open('politic_contents.pickle', 'rb') as f:
    news.extend(pickle.load(f))
with open('society_contents.pickle', 'rb') as f:
    news.extend(pickle.load(f))


In [5]:
len(id_news)

2238

In [6]:
len(news)

2238

In [7]:
import platform
from collections import Counter

import numpy as np

# tokenizer import
from konlpy.tag import Okt, Komoran, Hannanum, Kkma

#운영체제에 따라 mecab설치 방법이 다름.
if platform.system() == "Windows":
    try:
        from eunjeon import Mecab
    except:
        print("please install eunjeon module")
else:  # Ubuntu일 경우
    from konlpy.tag import Mecab

from typing import List, Callable, Union, Any, TypeVar, Tuple, Dict

from functools import partial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
import gensim
from gensim import corpora
import kss

from gensim.models import LdaModel


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [8]:
import warnings
warnings.filterwarnings(action='ignore')


In [9]:
import gensim
from gensim import corpora
import kss
import time
from konlpy.tag import Mecab
import random

In [10]:
print(len(news))
lines = ""

for i in news:
    lines = lines+i


2238


In [11]:
#불용어 사전에 "명" 추가
stop = "만,불,원,중,씨,헤럴드,전재,뿐,면,반,관련,기대,제기,우려,번,지적,배포,금지,말,건,분,회,간,내,수,거,게,명,직전,아,휴,아이구,아이쿠,아이고,어,나,우리,저희,따라,의해,을,를,에,의,가,으로,로,에게,뿐이다,의거하여,근거하여,입각하여,기준으로,예하면,예를 들면,예를 들자면,저,소인,소생,저희,지말고,하지마,하지마라,다른,물론,또한,그리고,비길수 없다,해서는 안된다,뿐만 아니라,만이 아니다,만은 아니다,막론하고,관계없이,그치지 않다,그러나,그런데,하지만,든간에,논하지 않다,따지지 않다,설사,비록,더라도,아니면,만 못하다,하는 편이 낫다,불문하고,향하여,향해서,향하다,쪽으로,틈타,이용하여,타다,오르다,제외하고,이 외에,이 밖에,하여야,비로소,한다면 몰라도,외에도,이곳,여기,부터,기점으로,따라서,할 생각이다,하려고하다,이리하여,그리하여,그렇게 함으로써,하지만,일때,할때,앞에서,중에서,보는데서,으로써,로써,까지,해야한다,일것이다,반드시,할줄알다,할수있다,할수있어,임에 틀림없다,한다면,등,등등,제,겨우,단지,다만,할뿐,딩동,댕그,대해서,대하여,대하면,훨씬,얼마나,얼마만큼,얼마큼,남짓,여,얼마간,약간,다소,좀,조금,다수,몇,얼마,지만,하물며,또한,그러나,그렇지만,하지만,이외에도,대해 말하자면,뿐이다,다음에,반대로,반대로 말하자면,이와 반대로,바꾸어서 말하면,바꾸어서 한다면,만약,그렇지않으면,까악,툭,딱,삐걱거리다,보드득,비걱거리다,꽈당,응당,해야한다,에 가서,각,각각,여러분,각종,각자,제각기,하도록하다,와,과,그러므로,그래서,고로,한 까닭에,하기 때문에,거니와,이지만,대하여,관하여,관한,과연,실로,아니나다를가,생각한대로,진짜로,한적이있다,하곤하였다,하,하하,허허,아하,거바,와,오,왜,어째서,무엇때문에,어찌,하겠는가,무슨,어디,어느곳,더군다나,하물며,더욱이는,어느때,언제,야,이봐,어이,여보시오,흐흐,흥,휴,헉헉,헐떡헐떡,영차,여차,어기여차,끙끙,아야,앗,아야,콸콸,졸졸,좍좍,뚝뚝,주룩주룩,솨,우르르,그래도,또,그리고,바꾸어말하면,바꾸어말하자면,혹은,혹시,답다,및,그에 따르는,때가 되어,즉,지든지,설령,가령,하더라도,할지라도,일지라도,지든지,몇,거의,하마터면,인젠,이젠,된바에야,된이상,만큼,어찌됏든,그위에,게다가,점에서 보아,비추어 보아,고려하면,하게될것이다,일것이다,비교적,좀,보다더,비하면,시키다,하게하다,할만하다,의해서,연이서,이어서,잇따라,뒤따라,뒤이어,결국,의지하여,기대여,통하여,자마자,더욱더,불구하고,얼마든지,마음대로,주저하지 않고,곧,즉시,바로,당장,하자마자,밖에 안된다,하면된다,그래,그렇지,요컨대,다시 말하자면,바꿔 말하면,즉,구체적으로,말하자면,시작하여,시초에,이상,허,헉,허걱,바와같이,해도좋다,해도된다,게다가,더구나,하물며,와르르,팍,퍽,펄렁,동안,이래,하고있었다,이었다,에서,로부터,까지,예하면,했어요,해요,함께,같이,더불어,마저,마저도,양자,모두,습니다,가까스로,하려고하다,즈음하여,다른,다른 방면으로,해봐요,습니까,했어요,말할것도 없고,무릎쓰고,개의치않고,하는것만 못하다,하는것이 낫다,매,매번,들,모,어느것,어느,로써,갖고말하자면,어디,어느쪽,어느것,어느해,어느 년도,라 해도,언젠가,어떤것,어느것,저기,저쪽,저것,그때,그럼,그러면,요만한걸,그래,그때,저것만큼,그저,이르기까지,할 줄 안다,할 힘이 있다,너,너희,당신,어찌,설마,차라리,할지언정,할지라도,할망정,할지언정,구토하다,게우다,토하다,메쓰겁다,옆사람,퉤,쳇,의거하여,근거하여,의해,따라,힘입어,그,다음,버금,두번째로,기타,첫번째로,나머지는,그중에서,견지에서,형식으로 쓰여,입장에서,위해서단지,의해되다,하도록시키다,뿐만아니라,반대로,전후,전자,앞의것,잠시,잠깐,하면서,그렇지만,다음에,그러한즉,그런즉,남들,아무거나,어찌하든지,같다,비슷하다,예컨대,이럴정도로,어떻게,만약,만일,위에서 서술한바와같이,인 듯하다,하지 않는다면,만약에,무엇,무슨,어느,어떤,아래윗,조차,한데,그럼에도 불구하고,여전히,심지어,까지도,조차도,하지 않도록,않기 위하여,때,시각,무렵,시간,동안,어때,어떠한,하여금,네,예,우선,누구,누가 알겠는가,아무도,줄은모른다,줄은 몰랏다,하는 김에,겸사겸사,하는바,그런 까닭에,한 이유는,그러니,그러니까,때문에,그,너희,그들,너희들,타인,것,것들,너,위하여,공동으로,동시에,하기 위하여,어찌하여,무엇때문에,붕붕,윙윙,나,우리,엉엉,휘익,윙윙,오호,아하,어쨋든,만 못하다,하기보다는,차라리,하는 편이 낫다,흐흐,놀라다,상대적으로 말하자면,마치,아니라면,쉿,그렇지 않으면,그렇지 않다면,안 그러면,아니었다면,하든지,아니면,이라면,좋아,알았어,하는것도,그만이다,어쩔수 없다,하나,일,일반적으로,일단,한켠으로는,오자마자,이렇게되면,이와같다면,전부,한마디,한항목,근거로,하기에,아울러,하지 않도록,않기 위해서,이르기까지,이 되다,로 인하여,까닭으로,이유만으로,이로 인하여,그래서,이 때문에,그러므로,그런 까닭에,알 수 있다,결론을 낼 수 있다,으로 인하여,있다,어떤것,관계가 있다,관련이 있다,연관되다,어떤것들,에 대해,이리하여,그리하여,여부,하기보다는,하느니,하면 할수록,운운,이러이러하다,하구나,하도다,다시말하면,다음으로,에 있다,에 달려 있다,우리,우리들,오히려,하기는한데,어떻게,어떻해,어찌됏어,어때,어째서,본대로,자,이,이쪽,여기,이것,이번,이렇게말하자면,이런,이러한,이와 같은,요만큼,요만한 것,얼마 안 되는 것,이만큼,이 정도의,이렇게 많은 것,이와 같다,이때,이렇구나,것과 같이,끼익,삐걱,따위,와 같은 사람들,부류의 사람들,왜냐하면,중의하나,오직,오로지,에 한하다,하기만 하면,도착하다,까지 미치다,도달하다,정도에 이르다,할 지경이다,결과에 이르다,관해서는,여러분,하고 있다,한 후,혼자,자기,자기집,자신,우에 종합한것과같이,총적으로 보면,총적으로 말하면,총적으로,대로 하다,으로서,참,그만이다,할 따름이다,쿵,탕탕,쾅쾅,둥둥,봐,봐라,아이야,아니,와아,응,아이,참나,년,월,일,령,영,일,이,삼,사,오,육,륙,칠,팔,구,이천육,이천칠,이천팔,이천구,하나,둘,셋,넷,다섯,여섯,일곱,여덟,아홉,령,영 "

In [12]:
news_doc = lines

stime = time.time()
text_list = kss.split_sentences(news_doc)
print(time.time()-stime)
k = []


23.5322048664093


In [13]:
stop_words = stop.split(',')
mecab = Mecab()
stime = time.time()
for i in news :
    sentences = mecab.nouns(i)
    result2 = []
    for word in sentences :
        if word not in stop_words:
            result2.append(word)
    k.append(result2)
print(time.time()-stime)
print(len(k))

9.217535972595215
2238


In [14]:
bigram = gensim.models.Phrases(k, min_count = 5, threshold = 0.000000000000000000001)

bigram_model = gensim.models.phrases.Phraser(bigram)

bigram_document = [bigram_model[nouns] for nouns in k]

for i in range(len(bigram_document)):
    if word in stop_words:
        bigram_document.drop(i)



In [15]:
id2word = corpora.Dictionary(bigram_document)
corpus = [id2word.doc2bow(doc) for doc in bigram_document]

In [16]:
from gensim.models import CoherenceModel

coherence_score=[]
t_min = 2
t_max = 20
for i in range(t_min,t_max):
    model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=i)
    coherence_model = CoherenceModel(model, texts=bigram_document, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model.get_coherence()
    print('n=',i,'\nCoherence Score: ', coherence_lda)
    coherence_score.append(coherence_lda)

n= 2 
Coherence Score:  0.2469734696739227
n= 3 
Coherence Score:  0.24356893275174465
n= 4 
Coherence Score:  0.34532882067334325
n= 5 
Coherence Score:  0.406883455408576
n= 6 
Coherence Score:  0.35640298538801146
n= 7 
Coherence Score:  0.4374950407594042
n= 8 
Coherence Score:  0.3535697985883468
n= 9 
Coherence Score:  0.40359749567336467
n= 10 
Coherence Score:  0.3807647543138698
n= 11 
Coherence Score:  0.40985544475137237
n= 12 
Coherence Score:  0.38672471489714494
n= 13 
Coherence Score:  0.39705139453931027
n= 14 
Coherence Score:  0.3707388647045734
n= 15 
Coherence Score:  0.3703246462512393
n= 16 
Coherence Score:  0.40094619322679403
n= 17 
Coherence Score:  0.37783463200882283
n= 18 
Coherence Score:  0.3562521494959913
n= 19 
Coherence Score:  0.4528329993456842


In [17]:
co_sc = np.array(coherence_score)
NUM_TOPICS = np.argmax(co_sc)+t_min

In [18]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=id2word, passes=15)
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.018*"선택_과목" + 0.013*"공통_과목" + 0.012*"평가" + 0.011*"모의_평가" + 0.011*"출제"')
(1, '0.008*"소송" + 0.007*"태양_반사광" + 0.007*"판단" + 0.006*"학생" + 0.006*"인정"')
(2, '0.014*"접종" + 0.012*"백신_접종" + 0.012*"백신" + 0.011*"아스트라_카" + 0.011*"예약"')
(3, '0.010*"회사" + 0.007*"지급" + 0.006*"지원" + 0.005*"경우" + 0.005*"퇴사"')
(4, '0.010*"수사" + 0.010*"앵커" + 0.008*"사건" + 0.008*"문제" + 0.007*"김영수"')
(5, '0.007*"중사" + 0.006*"데이터" + 0.006*"장_중사" + 0.004*"구속_영장" + 0.004*"유족_측"')
(6, '0.008*"홍콩" + 0.007*"차_씨" + 0.006*"진상_조사단" + 0.006*"자유" + 0.006*"한국"')
(7, '0.013*"국민_힘" + 0.011*"윤_전" + 0.009*"의원" + 0.009*"후보" + 0.008*"민주당"')
(8, '0.011*"공_수처" + 0.010*"네이버_블로그" + 0.008*"만_원" + 0.007*"수사" + 0.005*"조_교육감"')
(9, '0.009*"소년" + 0.006*"예비_명단" + 0.005*"온라인" + 0.005*"신규_확진" + 0.005*"기관"')
(10, '0.017*"인사" + 0.013*"박_장관" + 0.013*"김_총장" + 0.012*"법무부" + 0.011*"논의"')
(11, '0.005*"억_원" + 0.005*"개" + 0.005*"시장" + 0.004*"진행" + 0.004*"문화"')
(12, '0.017*"씨" + 0.011*"경찰" + 0.011*"부회장" + 0.009*"뒤" + 0.008*"사고"')
(13, '0.032*"택시_기사" + 0.021

In [19]:

def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob


In [20]:
lda_model = LdaModel(corpus, id2word=id2word, num_topics=NUM_TOPICS)

In [21]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(19,)
0.99999994
(19, 29218)
1.0


In [22]:
#lda_model.top_topics(corpus)

In [23]:
import pyLDAvis.gensim_models as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, id2word)

In [24]:
import pyLDAvis

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
pyLDAvis.display(prepared_data)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
for i, topic_list in enumerate(ldamodel[corpus]):
    print(id_news[i],'번째 문서의 topic 비율은',topic_list)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


4923 번째 문서의 topic 비율은 [(2, 0.016522488), (8, 0.2787815), (11, 0.08570639), (14, 0.19044392), (16, 0.11923762), (17, 0.30363062)]
4926 번째 문서의 topic 비율은 [(9, 0.03675894), (11, 0.5454111), (15, 0.06981046), (16, 0.2465733), (17, 0.08350216)]
4929 번째 문서의 topic 비율은 [(0, 0.019407548), (1, 0.022851028), (3, 0.058931794), (7, 0.025245272), (8, 0.044217773), (10, 0.034941845), (11, 0.383586), (15, 0.14613254), (16, 0.2342499), (18, 0.0209723)]
4932 번째 문서의 topic 비율은 [(0, 0.04092307), (1, 0.011379398), (2, 0.03707987), (8, 0.19035764), (10, 0.038182538), (17, 0.618419), (18, 0.052411463)]
4935 번째 문서의 topic 비율은 [(0, 0.012668582), (2, 0.0691339), (7, 0.012954861), (8, 0.1982108), (11, 0.06476817), (15, 0.045745634), (16, 0.05751798), (17, 0.5360598)]
4938 번째 문서의 topic 비율은 [(3, 0.05750359), (8, 0.19003895), (16, 0.24249066), (18, 0.5071619)]
4941 번째 문서의 topic 비율은 [(2, 0.046850804), (3, 0.035072986), (4, 0.022970688), (7, 0.32986572), (8, 0.16347606), (9, 0.0315875), (10, 0.019586857), (11, 0.1197994

5583 번째 문서의 topic 비율은 [(3, 0.24757516), (8, 0.51643574), (14, 0.19376872)]
5586 번째 문서의 topic 비율은 [(1, 0.035569046), (6, 0.06741813), (11, 0.7333666), (17, 0.15799667)]
5589 번째 문서의 topic 비율은 [(0, 0.011250513), (1, 0.05259614), (2, 0.017876782), (10, 0.2765904), (11, 0.13975811), (16, 0.071893536), (17, 0.13046716), (18, 0.29279515)]
5592 번째 문서의 topic 비율은 [(1, 0.010512565), (5, 0.33103532), (17, 0.13381742), (18, 0.5207939)]
5595 번째 문서의 topic 비율은 [(5, 0.14065287), (8, 0.034977216), (9, 0.043354403), (11, 0.0872115), (15, 0.035518195), (17, 0.07334415), (18, 0.5770751)]
5598 번째 문서의 topic 비율은 [(3, 0.022919418), (17, 0.95939183)]
5601 번째 문서의 topic 비율은 [(2, 0.11868892), (3, 0.07825296), (4, 0.44632432), (6, 0.052407894), (8, 0.046332765), (11, 0.018318746), (14, 0.052911546), (16, 0.029412434), (17, 0.11120627), (18, 0.036806308)]
5604 번째 문서의 topic 비율은 [(5, 0.052629374), (16, 0.020247675), (17, 0.40811363), (18, 0.5075436)]
5607 번째 문서의 topic 비율은 [(2, 0.2138551), (7, 0.055594552), (14, 0.6938

6153 번째 문서의 topic 비율은 [(2, 0.19911413), (6, 0.034760635), (8, 0.10915104), (9, 0.030025054), (14, 0.03070511), (15, 0.036212567), (16, 0.13831583), (18, 0.41749132)]
6156 번째 문서의 topic 비율은 [(1, 0.12961821), (3, 0.028258383), (6, 0.02058509), (15, 0.11165151), (16, 0.10901951), (17, 0.5874487), (18, 0.011026592)]
6159 번째 문서의 topic 비율은 [(6, 0.6038868), (11, 0.11146261), (15, 0.10658251), (16, 0.052919004), (18, 0.12275086)]
6162 번째 문서의 topic 비율은 [(1, 0.6073168), (11, 0.122721285), (15, 0.21911448), (17, 0.04466458)]
6165 번째 문서의 topic 비율은 [(1, 0.71542215), (16, 0.08077491), (17, 0.08431601), (18, 0.11454407)]
6168 번째 문서의 topic 비율은 [(1, 0.05036515), (2, 0.039477613), (4, 0.016171623), (6, 0.52476984), (8, 0.09546995), (11, 0.044726618), (13, 0.050050575), (18, 0.17179458)]
6171 번째 문서의 topic 비율은 [(2, 0.0718563), (6, 0.029874941), (7, 0.53799397), (11, 0.17827082), (13, 0.025325665), (17, 0.15344127)]
6174 번째 문서의 topic 비율은 [(2, 0.018569738), (3, 0.05245597), (6, 0.040223397), (7, 0.023240486)

6736 번째 문서의 topic 비율은 [(2, 0.0836612), (3, 0.25148427), (5, 0.059563793), (7, 0.32313898), (11, 0.2693813)]
6738 번째 문서의 topic 비율은 [(1, 0.027047152), (9, 0.10583255), (10, 0.22828026), (11, 0.34777343), (12, 0.08196885), (15, 0.19611016)]
6740 번째 문서의 topic 비율은 [(0, 0.03344866), (2, 0.28474116), (6, 0.03156644), (11, 0.55045295), (17, 0.08288231)]
6742 번째 문서의 topic 비율은 [(1, 0.15351523), (5, 0.015754528), (9, 0.13351807), (11, 0.21908867), (15, 0.30684432), (16, 0.08940982), (17, 0.073905595)]
6744 번째 문서의 topic 비율은 [(1, 0.03460289), (3, 0.03984193), (6, 0.5284537), (11, 0.07720722), (15, 0.20731425), (16, 0.101332515)]
6746 번째 문서의 topic 비율은 [(6, 0.022799132), (9, 0.121844105), (10, 0.42003226), (11, 0.13074766), (15, 0.23135653), (16, 0.04363357), (18, 0.021655286)]
6748 번째 문서의 topic 비율은 [(5, 0.27605352), (10, 0.092762664), (11, 0.20531899), (17, 0.07867442), (18, 0.32725856)]
6750 번째 문서의 topic 비율은 [(0, 0.019893782), (1, 0.021721749), (4, 0.0105425), (5, 0.034709558), (6, 0.049191598), (1

...

6260 번째 문서의 topic 비율은 [(4, 0.046561692), (7, 0.2886522), (11, 0.018068442), (12, 0.61629003), (14, 0.0242787)]
6263 번째 문서의 topic 비율은 [(7, 0.7576343), (11, 0.019995749), (12, 0.09821016), (14, 0.11797434)]
6266 번째 문서의 topic 비율은 [(0, 0.34316203), (4, 0.4276605), (5, 0.18039659), (7, 0.04208288)]
6269 번째 문서의 topic 비율은 [(5, 0.82835764), (15, 0.1367957), (16, 0.020800704)]
6272 번째 문서의 topic 비율은 [(4, 0.1769997), (7, 0.38482678), (8, 0.03223982), (10, 0.275917), (14, 0.11581893)]
6275 번째 문서의 topic 비율은 [(5, 0.9311555), (11, 0.027464824), (17, 0.02416757)]
6278 번째 문서의 topic 비율은 [(4, 0.09808696), (5, 0.8483062), (10, 0.045803938)]
6281 번째 문서의 topic 비율은 [(7, 0.58784425), (14, 0.39554685)]
6284 번째 문서의 topic 비율은 [(4, 0.4715799), (5, 0.08209253), (7, 0.14803128), (10, 0.28215212)]
6287 번째 문서의 topic 비율은 [(13, 0.82883686), (18, 0.1493271)]
6290 번째 문서의 topic 비율은 [(2, 0.42345518), (5, 0.04577958), (6, 0.063313775), (11, 0.21714061), (15, 0.09045388), (17, 0.06306158), (18, 0.085273586)]
6293 번째 문서의 topi

5242 번째 문서의 topic 비율은 [(0, 0.023446545), (1, 0.106818505), (2, 0.031478256), (3, 0.081727095), (11, 0.03233161), (15, 0.042645875), (16, 0.28050154), (17, 0.39763373)]
5245 번째 문서의 topic 비율은 [(8, 0.293295), (18, 0.6995388)]
5248 번째 문서의 topic 비율은 [(2, 0.63709676), (8, 0.33736205), (15, 0.018456882)]
5251 번째 문서의 topic 비율은 [(8, 0.21685518), (9, 0.2274908), (17, 0.5500327)]
5254 번째 문서의 topic 비율은 [(0, 0.44980362), (2, 0.05765835), (5, 0.029874815), (16, 0.4523774)]
5257 번째 문서의 topic 비율은 [(2, 0.6191744), (4, 0.020706499), (8, 0.27956182), (14, 0.03223465), (17, 0.04433036)]
5260 번째 문서의 topic 비율은 [(8, 0.2444556), (9, 0.041879497), (11, 0.15646619), (15, 0.55273175)]
5263 번째 문서의 topic 비율은 [(2, 0.14018576), (8, 0.834953)]
5266 번째 문서의 topic 비율은 [(1, 0.010396036), (2, 0.17915548), (6, 0.4190598), (8, 0.2663086), (16, 0.013378415), (17, 0.10612545)]
5269 번째 문서의 topic 비율은 [(2, 0.712883), (8, 0.25215507), (9, 0.029723795)]
5272 번째 문서의 topic 비율은 [(0, 0.8385791), (4, 0.02013543), (8, 0.13805667)]
5275 

5719 번째 문서의 topic 비율은 [(2, 0.012775773), (5, 0.62158006), (8, 0.04208666), (9, 0.019092387), (18, 0.29447997)]
5722 번째 문서의 topic 비율은 [(2, 0.036166795), (5, 0.33505964), (18, 0.61618286)]
5725 번째 문서의 topic 비율은 [(7, 0.9210208), (18, 0.064058654)]
5728 번째 문서의 topic 비율은 [(0, 0.12918659), (1, 0.44920483), (2, 0.038809065), (11, 0.010140245), (15, 0.04030769), (17, 0.22646189), (18, 0.10187958)]
5731 번째 문서의 topic 비율은 [(0, 0.08182433), (1, 0.24504815), (13, 0.024175618), (17, 0.38609427), (18, 0.25878134)]
5734 번째 문서의 topic 비율은 [(0, 0.10507062), (1, 0.4328021), (2, 0.054357637), (15, 0.02945619), (17, 0.15777062), (18, 0.21755096)]
5737 번째 문서의 topic 비율은 [(1, 0.016314676), (4, 0.011130326), (7, 0.22602461), (16, 0.54644597), (18, 0.19723642)]
5740 번째 문서의 topic 비율은 [(1, 0.6537428), (3, 0.05600777), (6, 0.03298621), (8, 0.052518897), (15, 0.014174184), (17, 0.05547705), (18, 0.12787609)]
5743 번째 문서의 topic 비율은 [(4, 0.5392767), (5, 0.3263813), (18, 0.123157606)]
5746 번째 문서의 topic 비율은 [(0, 0.035305

6508 번째 문서의 topic 비율은 [(2, 0.23877059), (3, 0.27905977), (6, 0.05955269), (8, 0.110507995), (9, 0.09345132), (15, 0.07934966), (17, 0.13504823)]
6511 번째 문서의 topic 비율은 [(0, 0.10562331), (3, 0.0432858), (4, 0.04877087), (5, 0.0100715235), (6, 0.035899006), (7, 0.07559525), (10, 0.1117207), (11, 0.07746018), (12, 0.033424165), (13, 0.06462016), (15, 0.25804695), (16, 0.022987148), (18, 0.11080497)]
6514 번째 문서의 topic 비율은 [(1, 0.17533788), (2, 0.016974319), (4, 0.010784975), (6, 0.037353348), (8, 0.03994346), (9, 0.053760823), (11, 0.06243167), (13, 0.027597014), (14, 0.028888818), (15, 0.0425019), (17, 0.50277954)]
6517 번째 문서의 topic 비율은 [(1, 0.0296636), (11, 0.1497356), (15, 0.040056016), (16, 0.71922386), (17, 0.055599693)]
6519 번째 문서의 topic 비율은 [(0, 0.031704966), (1, 0.03530694), (2, 0.013094848), (3, 0.3473893), (9, 0.23674853), (11, 0.13601987), (16, 0.14184055), (17, 0.05475361)]
6521 번째 문서의 topic 비율은 [(4, 0.018816268), (6, 0.057420492), (9, 0.038624696), (10, 0.02986845), (12, 0.8425

6966 번째 문서의 topic 비율은 [(1, 0.9941873)]
6967 번째 문서의 topic 비율은 [(7, 0.9529979), (17, 0.036556836)]
6968 번째 문서의 topic 비율은 [(3, 0.914356), (11, 0.037404567), (15, 0.020963766)]
6969 번째 문서의 topic 비율은 [(12, 0.91114014), (15, 0.06530247)]
6970 번째 문서의 topic 비율은 [(0, 0.9785921), (18, 0.011478319)]
6971 번째 문서의 topic 비율은 [(0, 0.010669198), (8, 0.9726521)]
6972 번째 문서의 topic 비율은 [(2, 0.16603085), (3, 0.4603316), (7, 0.050726533), (12, 0.2853031)]
6973 번째 문서의 topic 비율은 [(2, 0.2509117), (3, 0.3737292), (7, 0.1570096), (12, 0.18401481)]
6974 번째 문서의 topic 비율은 [(4, 0.06445442), (6, 0.06370343), (7, 0.018468568), (13, 0.8486997)]
6975 번째 문서의 topic 비율은 [(10, 0.013212232), (17, 0.95616275)]
6976 번째 문서의 topic 비율은 [(2, 0.50059944), (9, 0.45019576), (17, 0.03247454)]
6977 번째 문서의 topic 비율은 [(0, 0.04489111), (1, 0.035034925), (5, 0.032992475), (8, 0.08858591), (11, 0.78860664)]
6978 번째 문서의 topic 비율은 [(12, 0.7094942), (15, 0.27533597)]
6979 번째 문서의 topic 비율은 [(6, 0.57772124), (9, 0.31554687), (10, 0.014885809), (

In [27]:
pp = [[]]*(NUM_TOPICS+1)#NO Topic0
for i in range(NUM_TOPICS):
    pp[i] = prepared_data.sorted_terms(topic = i,_lambda=0.6).iloc[:20]

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
#index만 추출하기(index가 corpus의 index랑 동일함 ㅇㅇ)
idx_topic = [[]]*(NUM_TOPICS+1)
for i in range(1,len(pp)-1):
    idx_topic[i] = pp[i].iloc[:20].index

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:

#topic별 clustering을 위한 topic_docs 초기화(list[[]]*n 이런 식으로 하면 안됨)
topic_docs = []
topic_docs_save = []
for i in range(NUM_TOPICS+1):
    topic_docs.append([])
    topic_docs_save.append([])

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
#append doc_index in the topic_docs cluster
for i, topic_list in enumerate(ldamodel[corpus]):
    topic_list.sort(reverse=True,key = lambda element:element[1])
    n = topic_list[0][0] + 1
    topic_docs[n].append([i,topic_list[0][1]])


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
#append doc_index in the topic_docs cluster
for i, topic_list in enumerate(ldamodel[corpus]):
    topic_list.sort(reverse=True,key = lambda element:element[1])
    n = topic_list[0][0] + 1
    topic_docs_save[n].append([id_news[i],topic_list[0][1]])


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#topic_docs_save[2]
#반환해줄 실제 news index와 가중치 저장해둔 list

In [32]:
for i in topic_docs:
    i.sort(reverse=True,key = lambda element:element[1])


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
topic_cluster = []
for i in range(NUM_TOPICS+1):
    topic_cluster.append("")
for i in range(1,NUM_TOPICS+1):
    for j in topic_docs[i]:
        topic_cluster[i] = topic_cluster[i]+news[j[0]]
    

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
for i, topic_list in enumerate(ldamodel[corpus]):
    topic_list.sort(reverse=True,key= lambda element:element[1])
    print(i,'번째 문서의 topic 비율은',topic_list)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0 번째 문서의 topic 비율은 [(17, 0.30363813), (8, 0.27878463), (14, 0.190445), (16, 0.11924236), (11, 0.08567301), (2, 0.016539445)]
1 번째 문서의 topic 비율은 [(11, 0.5454071), (16, 0.246571), (17, 0.08351024), (15, 0.069808036), (9, 0.03676051)]
2 번째 문서의 topic 비율은 [(11, 0.3835879), (16, 0.23425162), (15, 0.14613482), (3, 0.058937546), (8, 0.04421693), (10, 0.034941923), (7, 0.025245188), (1, 0.022855394), (18, 0.020973086), (0, 0.01940911)]
3 번째 문서의 topic 비율은 [(17, 0.6184162), (8, 0.19035704), (18, 0.052408647), (0, 0.040923283), (10, 0.038182333), (2, 0.037078936), (1, 0.011381729)]
4 번째 문서의 topic 비율은 [(17, 0.5360606), (8, 0.1982113), (2, 0.069133185), (11, 0.06476755), (16, 0.057516146), (15, 0.04574739), (7, 0.0129548665), (0, 0.012668731)]
5 번째 문서의 topic 비율은 [(18, 0.5071598), (16, 0.24249348), (8, 0.19004157), (3, 0.057500247)]
6 번째 문서의 topic 비율은 [(7, 0.32984266), (8, 0.16374594), (11, 0.12043344), (17, 0.09369833), (18, 0.06116765), (2, 0.046596006), (16, 0.039017525), (3, 0.035438437), (9, 0.0

218 번째 문서의 topic 비율은 [(17, 0.5740632), (11, 0.14090389), (3, 0.11687275), (8, 0.11028925), (14, 0.028214104), (6, 0.023693802)]
219 번째 문서의 topic 비율은 [(17, 0.5052861), (7, 0.11257293), (6, 0.08635043), (5, 0.080631666), (10, 0.07501479), (11, 0.0554192), (3, 0.045728173), (1, 0.034392316)]
220 번째 문서의 topic 비율은 [(8, 0.5164337), (3, 0.24754435), (14, 0.19380158)]
221 번째 문서의 topic 비율은 [(11, 0.7333671), (17, 0.1579953), (6, 0.06741814), (1, 0.035569996)]
222 번째 문서의 topic 비율은 [(18, 0.29279295), (10, 0.27658948), (11, 0.1397544), (17, 0.13047937), (16, 0.071892194), (1, 0.052595664), (2, 0.017873153), (0, 0.011250665)]
223 번째 문서의 topic 비율은 [(18, 0.52079576), (5, 0.33103752), (17, 0.13381658), (1, 0.010509335)]
224 번째 문서의 topic 비율은 [(18, 0.5770761), (5, 0.14065333), (11, 0.0872115), (17, 0.07334115), (9, 0.04335736), (15, 0.035517987), (8, 0.034976017)]
225 번째 문서의 topic 비율은 [(17, 0.959358), (3, 0.022914862)]
226 번째 문서의 topic 비율은 [(4, 0.4463249), (2, 0.11868786), (17, 0.11121514), (3, 0.0782496

418 번째 문서의 topic 비율은 [(15, 0.357614), (8, 0.20493826), (16, 0.17459926), (18, 0.088556565), (2, 0.04886562), (1, 0.04004651), (11, 0.037625577), (17, 0.027987726)]
419 번째 문서의 topic 비율은 [(11, 0.7629826), (15, 0.1315001), (2, 0.026750648), (8, 0.024045886), (5, 0.022435222), (17, 0.019401029)]
420 번째 문서의 topic 비율은 [(3, 0.9233998), (1, 0.07263723)]
421 번째 문서의 topic 비율은 [(12, 0.49458146), (18, 0.11423747), (17, 0.07865249), (2, 0.06786971), (5, 0.06182781), (15, 0.045299053), (11, 0.041779567), (16, 0.039435036), (3, 0.024628988), (9, 0.015910633), (13, 0.014065339)]
422 번째 문서의 topic 비율은 [(18, 0.3151613), (15, 0.25063127), (17, 0.15195386), (16, 0.11795909), (11, 0.060225226), (1, 0.04053268), (12, 0.022268), (5, 0.014200486), (3, 0.0125632705)]
423 번째 문서의 topic 비율은 [(16, 0.99435407)]
424 번째 문서의 topic 비율은 [(4, 0.44009632), (9, 0.1564195), (11, 0.11322106), (3, 0.08503514), (15, 0.063361496), (17, 0.05155188), (18, 0.026854284), (16, 0.020752238), (1, 0.019386442), (0, 0.015235526)]
425 번째 

662 번째 문서의 topic 비율은 [(18, 0.7169387), (15, 0.07472957), (17, 0.061777055), (10, 0.060947586), (2, 0.03882385), (8, 0.029365147), (13, 0.012890311)]
663 번째 문서의 topic 비율은 [(11, 0.54803276), (6, 0.31372103), (15, 0.066081814), (4, 0.06568212)]
664 번째 문서의 topic 비율은 [(3, 0.43506423), (4, 0.11662138), (15, 0.0912494), (7, 0.07400784), (11, 0.053040516), (2, 0.05258917), (1, 0.04302153), (9, 0.03561545), (17, 0.035515692), (6, 0.014634704), (8, 0.012891949)]
665 번째 문서의 topic 비율은 [(11, 0.2029607), (14, 0.15388536), (1, 0.12455628), (18, 0.11996493), (4, 0.11159747), (3, 0.081813164), (15, 0.070919424), (17, 0.06549047), (5, 0.043624423), (16, 0.012393313), (8, 0.010046704)]
666 번째 문서의 topic 비율은 [(17, 0.479946), (16, 0.1414679), (3, 0.13906895), (18, 0.09103748), (8, 0.085652635), (10, 0.026590524), (4, 0.026308337)]
667 번째 문서의 topic 비율은 [(18, 0.3073862), (17, 0.26336956), (11, 0.23169607), (14, 0.07576523), (15, 0.0336728), (16, 0.030243516), (5, 0.027968299), (1, 0.023651104)]
668 번째 문서의 top

...

1052 번째 문서의 topic 비율은 [(7, 0.836122), (15, 0.061086807), (10, 0.04653823), (3, 0.019076277), (12, 0.012240934), (14, 0.010705364)]
1053 번째 문서의 topic 비율은 [(2, 0.3089202), (3, 0.30585724), (15, 0.1602655), (7, 0.0906767), (9, 0.06616195), (14, 0.037029434), (0, 0.027021367)]
1054 번째 문서의 topic 비율은 [(0, 0.412084), (17, 0.13714813), (15, 0.12173751), (2, 0.089073464), (13, 0.065010786), (4, 0.063749775), (16, 0.056252938), (5, 0.05032774)]
1055 번째 문서의 topic 비율은 [(18, 0.77424985), (0, 0.16712336), (15, 0.047901113)]
1056 번째 문서의 topic 비율은 [(17, 0.6161185), (0, 0.21367615), (15, 0.11278619), (5, 0.021479672), (10, 0.017977083), (6, 0.015284227)]
1057 번째 문서의 topic 비율은 [(7, 0.83465946), (17, 0.06251617), (9, 0.042611), (16, 0.03005319), (10, 0.025274888)]
1058 번째 문서의 topic 비율은 [(7, 0.5969453), (18, 0.17884089), (16, 0.10249035), (4, 0.03748817), (1, 0.024303196), (9, 0.021229554), (17, 0.01913037), (8, 0.01469146)]
1059 번째 문서의 topic 비율은 [(4, 0.41219082), (15, 0.22914222), (3, 0.17127237), (10, 0

1245 번째 문서의 topic 비율은 [(2, 0.6351746), (8, 0.33779228), (10, 0.011255154)]
1246 번째 문서의 topic 비율은 [(8, 0.25238413), (15, 0.22353405), (7, 0.2061955), (12, 0.16159149), (11, 0.13286762), (4, 0.019153835)]
1247 번째 문서의 topic 비율은 [(8, 0.3207247), (9, 0.25636274), (15, 0.17654143), (1, 0.064968236), (2, 0.060148083), (11, 0.047251977), (4, 0.046853725), (3, 0.022420133)]
1248 번째 문서의 topic 비율은 [(8, 0.6970214), (17, 0.2739734), (9, 0.01908473)]
1249 번째 문서의 topic 비율은 [(16, 0.18940255), (4, 0.15905148), (10, 0.13350539), (9, 0.09641116), (12, 0.07512194), (3, 0.07366666), (6, 0.072603814), (11, 0.06052145), (8, 0.05095236), (17, 0.036324814), (15, 0.0284024), (1, 0.019036973)]
1250 번째 문서의 topic 비율은 [(8, 0.3134025), (12, 0.16424125), (1, 0.12645379), (4, 0.12086752), (15, 0.08592823), (13, 0.056732055), (14, 0.036677368), (3, 0.028677864), (5, 0.028414143), (11, 0.028320953)]
1251 번째 문서의 topic 비율은 [(8, 0.87697977), (2, 0.09503094)]
1252 번째 문서의 topic 비율은 [(15, 0.32769376), (11, 0.2860106), (8, 0.2

1478 번째 문서의 topic 비율은 [(18, 0.4227178), (15, 0.15518612), (7, 0.12587754), (9, 0.07512188), (4, 0.06800046), (11, 0.05170068), (12, 0.02196045), (2, 0.021086613), (1, 0.019863231), (3, 0.016143437), (10, 0.012197797)]
1479 번째 문서의 topic 비율은 [(18, 0.3875998), (13, 0.26635104), (12, 0.2569444), (15, 0.08050645)]
1480 번째 문서의 topic 비율은 [(16, 0.59435195), (18, 0.16530575), (12, 0.0866105), (4, 0.08481964), (11, 0.028560318), (5, 0.020844523), (17, 0.01720703)]
1481 번째 문서의 topic 비율은 [(18, 0.35488552), (9, 0.29221314), (2, 0.27724338), (13, 0.03212838), (10, 0.020120747), (12, 0.017691836)]
1482 번째 문서의 topic 비율은 [(18, 0.45271495), (13, 0.19439654), (8, 0.1916547), (12, 0.07679726), (4, 0.04041992), (11, 0.03619854)]
1483 번째 문서의 topic 비율은 [(18, 0.993461)]
1484 번째 문서의 topic 비율은 [(8, 0.3720149), (18, 0.33002833), (12, 0.14973249), (14, 0.097148195), (16, 0.02745392), (1, 0.01804862)]
1485 번째 문서의 topic 비율은 [(18, 0.67590517), (2, 0.28452003), (3, 0.036047235)]
1486 번째 문서의 topic 비율은 [(0, 0.7824649),

1707 번째 문서의 topic 비율은 [(11, 0.27709964), (15, 0.27491868), (17, 0.1632354), (0, 0.14554743), (16, 0.07175163), (18, 0.059640586)]
1708 번째 문서의 topic 비율은 [(2, 0.522668), (15, 0.14391857), (12, 0.10869818), (4, 0.06626957), (9, 0.0542262), (11, 0.037698798), (1, 0.032469273), (18, 0.019169502), (6, 0.011182716)]
1709 번째 문서의 topic 비율은 [(12, 0.52992404), (11, 0.20991583), (9, 0.13978831), (15, 0.11669329)]
1710 번째 문서의 topic 비율은 [(14, 0.9697778), (6, 0.021445572)]
1711 번째 문서의 topic 비율은 [(7, 0.64349943), (15, 0.20657562), (16, 0.07212423), (18, 0.039922908), (9, 0.021807197), (8, 0.012462774)]
1712 번째 문서의 topic 비율은 [(13, 0.23555969), (2, 0.16978662), (15, 0.14641704), (3, 0.11346329), (7, 0.057913933), (8, 0.05389019), (9, 0.052439474), (1, 0.047986824), (12, 0.04553964), (18, 0.029351132), (11, 0.024041742), (16, 0.011033848), (14, 0.011027143)]
1713 번째 문서의 topic 비율은 [(11, 0.45260736), (15, 0.32252148), (2, 0.09353303), (16, 0.06794445), (18, 0.05816029)]
1714 번째 문서의 topic 비율은 [(15, 0.807967

1895 번째 문서의 topic 비율은 [(11, 0.3199503), (9, 0.19168521), (16, 0.14356779), (6, 0.123659976), (4, 0.067293145), (12, 0.04523688), (8, 0.032288533), (2, 0.03153838), (7, 0.019078273), (13, 0.011290763)]
1896 번째 문서의 topic 비율은 [(11, 0.42712614), (12, 0.3065625), (15, 0.0884783), (9, 0.08158351), (13, 0.07100946), (14, 0.021430489)]
1897 번째 문서의 topic 비율은 [(3, 0.8097954), (4, 0.06328075), (16, 0.056550235), (14, 0.035852045), (15, 0.013564605)]
1898 번째 문서의 topic 비율은 [(2, 0.99002266)]
1899 번째 문서의 topic 비율은 [(2, 0.9706189), (15, 0.014258538)]
1900 번째 문서의 topic 비율은 [(15, 0.28210127), (14, 0.20803156), (16, 0.17222434), (4, 0.15115519), (11, 0.10479022), (10, 0.059202466), (8, 0.014470885)]
1901 번째 문서의 topic 비율은 [(1, 0.36782196), (15, 0.25182128), (11, 0.09723206), (16, 0.09433102), (9, 0.08193824), (3, 0.043009136), (5, 0.033622067), (13, 0.025571413)]
1902 번째 문서의 topic 비율은 [(16, 0.94675475), (11, 0.036929112), (5, 0.010578573)]
1903 번째 문서의 topic 비율은 [(14, 0.39016303), (9, 0.26407307), (4, 0.14

In [35]:
topic_cluster_sentences = []
tcs = []
for i in range(NUM_TOPICS+1):
    tcs.append([])
    topic_cluster_sentences.append([])

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
for i in range(1,NUM_TOPICS+1):
    mecab = Mecab()
    stop_words = stop.split(',')
    topic_cluster_sentences[i] = kss.split_sentences(topic_cluster[i])
    for j in topic_cluster_sentences[i]:
        sen_word = []
        sentences = mecab.nouns(j)
        for word in sentences :
            if word not in stop_words:
                sen_word.append(word)
        tcs[i].append(sen_word)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
bigram_docs = []
for i in range(NUM_TOPICS+1):
    bigram_docs.append([])
#construct bigram_docs --> use the model made before
for i in range(1,len(tcs)):
    bigram_docs[i] = [bigram_model[nouns] for nouns in tcs[i]]

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
corpus_docs = []
for i in range(NUM_TOPICS+1):
    corpus_docs.append([])
for i in range(1,NUM_TOPICS+1):    
    corpus_docs[i] = [id2word.doc2bow(doc) for doc in bigram_docs[i]]

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
corp_doc_ref = []
for i in range(NUM_TOPICS+1):
    corp_doc_ref.append([])


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
stime = time.time()

for i in range(1,NUM_TOPICS+1):
    if len(corpus_docs[i]) is not 0:
        for j in range(len(corpus_docs[i])):
            corp_doc_ref[i].append([])
            for k in range(len(corpus_docs[i][j])):
                a = corpus_docs[i][j][k][0]
                corp_doc_ref[i][j].append(a)

print(time.time()-stime)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.13202500343322754


In [41]:
#키워드 포함된 문장만 추출
stime = time.time()
corp_doc_topic = []
for i in range(NUM_TOPICS+1):
    corp_doc_topic.append([])
for i in range(1,len(corp_doc_ref)):
    for j in range(len(corp_doc_ref[i])):
        if len(set(idx_topic[i-1]).intersection(corp_doc_ref[i][j])) is not 0:
            corp_doc_topic[i].append(bigram_docs[i][j])

print(time.time()-stime)

0.05469179153442383


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


corp_doc_topic[N][M] : N개의 topic, M개의 문장 키워드 포함한 문장만 뽑아낸 것

In [42]:
def vectorize_sents(corp_doc_topic = corp_doc_topic, min_count=2, tokenizer="mecab", noun=True):
   

    vectorizer = CountVectorizer(tokenizer=lambda x: x,lowercase=False)

    vec = vectorizer.fit_transform(corp_doc_topic)
    vocab_idx = vectorizer.vocabulary_
    idx_vocab = {idx: vocab for vocab, idx in vocab_idx.items()}
    
    return vec, vocab_idx, idx_vocab

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
def word_similarity_matrix(x, min_sim=0.3):
    sim_mat = 1 - pairwise_distances(x.T, metric="cosine")
    sim_mat[np.where(sim_mat <= min_sim)] = 0

    return sim_mat

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
def word_graph(
    corp_doc_topic = corp_doc_topic,
    min_count=2,
    min_sim=0.3,
    tokenizer="mecab",
    noun=True,
):

    mat, vocab_idx, idx_vocab = vectorize_sents(
        corp_doc_topic,min_count=min_count, tokenizer=tokenizer, noun=noun
    )

    mat = word_similarity_matrix(mat, min_sim=min_sim)

    return mat, vocab_idx, idx_vocab

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
def pagerank(x: np.ndarray, df=0.85, max_iter=50, method="iterative"):

    assert 0 < df < 1

    A = normalize(mat, axis=0, norm="l1")
    N = np.ones(A.shape[0]) / A.shape[0]

    if method == "iterative":
        R = np.ones(A.shape[0])
        for _ in range(max_iter):
            R = df * np.matmul(A, R) + (1 - df) * N
    elif method == "algebraic":
        R = np.linalg.inv((I - df * A))
        R = np.matmul(R, (1 - df) * N)

    return R

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
keywords = []

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
for i in range(len(corp_doc_topic)):
    if len(corp_doc_topic[i])!= 0:
        mat,vocab_idx, idx_vocab = word_graph(corp_doc_topic[i])
        R = pagerank(mat,method ="iterative")
        topk = 10
        idxs = R.argsort()[-topk:]
        keywords.append([(idx, R[idx], idx_vocab[idx]) for idx in reversed(idxs)])
        #keyword = [(R[idx]) for idx in reversed(idxs)]


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
keywords

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(1295, 0.0006654039412149883, '생활'),
  (1050, 0.0006426677699507051, '법원'),
  (3156, 0.0006346759352618038, '힐튼_서울'),
  (1102, 0.0006346759352618038, '복지'),
  (1111, 0.0006346759352618038, '봉사_활동'),
  (1110, 0.0006346759352618038, '봉봉'),
  (1109, 0.0006346759352618038, '봉고'),
  (1108, 0.0006346759352618038, '봄'),
  (1107, 0.0006346759352618038, '본회의'),
  (1106, 0.0006346759352618038, '본인')],
 [(1724, 0.001561661846578161, '예방_접종'),
  (1801, 0.001561113030407301, '외'),
  (1233, 0.0015548692715150441, '사전_예약'),
  (2914, 0.0015531231643263928, '한국_마이크'),
  (2767, 0.0015531231643263928, '크레디트_조합'),
  (1387, 0.0015441575049095655, '센터'),
  (961, 0.0015435823388516075, '발견'),
  (2613, 0.0015435823388516075, '체결_협약'),
  (1015, 0.0015435823388516075, '배달_앱'),
  (2109, 0.0015435823388516075, '잔량')],
 [(2317, 0.0008243706104168057, '총괄'),
  (2602, 0.0008243706104168057, '해당'),
  (1481, 0.0008243706104168057, '역할'),
  (1620, 0.00082293024963531, '원'),
  (23, 0.0008046679144755879, '가운데'),
  (50

In [49]:
for i in keywords:
    print(i[0])

(1295, 0.0006654039412149883, '생활')
(1724, 0.001561661846578161, '예방_접종')
(2317, 0.0008243706104168057, '총괄')
(3009, 0.0008293066532886789, '조사')
(2892, 0.001233055967303463, '중사')
(670, 0.0023539725687392503, '죽음')
(4987, 0.0006950460378963683, '행사')
(1575, 0.0016234435014471972, '입사_만')
(1298, 0.001288178666609656, '올해')
(516, 0.0028600045382783557, '절차')
(109, 0.001159701498362669, '검찰')
(1243, 0.0017652006869657466, '최근')
(284, 0.005123487726645096, '택시_기사')
(590, 0.001987311593215722, '흔적')
(888, 0.0009628248686309292, '내용')
(150, 0.0011580724279607023, '구체')
(587, 0.0012257081411531346, '단체')
(42, 0.016754499156691777, '여성')


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
